In [48]:
import mxnet as mx

In [49]:
from mxnet import ndarray as nd

In [50]:
from collections import namedtuple

In [51]:
import numpy as np

In [115]:
import caffe

In [118]:
from caffe import layers, params

In [52]:
data = nd.random.uniform(shape=(1,3,20,20))

In [53]:
Batch = namedtuple('Batch', ['data'])

In [54]:
ctx = mx.cpu()

In [55]:
sym, arg, aux = mx.model.load_checkpoint('test_net', epoch=1)

获得所有中间输出

In [69]:
args = sym.get_internals().list_outputs()

In [96]:
internals = sym.get_internals()

In [100]:
conv1 = internals[2]

In [102]:
group = mx.symbol.Group([conv1, sym])

In [104]:
mod = mx.mod.Module(symbol=group, context=ctx)

mod.bind(for_training=False, data_shapes=[('data', (1,3,20,20))])

mod.set_params(arg_params=arg, aux_params=aux)

mod.forward(Batch([data]))

prob = mod.get_outputs()

In [108]:
print internals

<Symbol group [data, conv1_weight, conv1, conv1_batchnorm_gamma, conv1_batchnorm_beta, conv1_batchnorm_moving_mean, conv1_batchnorm_moving_var, conv1_batchnorm, conv1_relu, conv2_dw_weight, conv2_dw, conv2_dw_batchnorm_gamma, conv2_dw_batchnorm_beta, conv2_dw_batchnorm_moving_mean, conv2_dw_batchnorm_moving_var, conv2_dw_batchnorm, conv2_weight, conv2, flatten0, fullyconnected0_weight, fullyconnected0_bias, fullyconnected0, softmax_label, softmax]>


In [136]:
internals[15]

<Symbol conv2_dw_batchnorm>

In [142]:
internals[17]

<Symbol conv2>

In [150]:
internals[21]

<Symbol fullyconnected0>